Data cleanup in another file

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import requests_oauthlib

FileName = '/content/drive/MyDrive/SpartaHack8/Training Images'
df = pd.read_csv('final_training.csv')

X = pd.get_dummies(df.drop(['image', 'image_ID' , 'num_potholes' , 'x_min' , 'y_min' , 'width' , 'height'], axis = 1))
y = df['image'].apply(lambda x : 1 if X == 'Yes' else 0)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2)

X_train.head()
y_train.head()


SpartaHack8 Project for social good: Convolutional Neural Network to identify potholes using image recognition.

In [ ]:
!pip install streamlit

In [ ]:
import tensorflow
import keras

from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense
from sklearn.metrics import accuracy_score

model = Sequential()
model.add(Dense(units=32, activation='relu', input_dim=len(X_train.columns)))
model.add(Dense(units=64, activation='relu'))
model.add(Dense(units=1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='sgd', metrics='accuracy')

model.fit(X_train, y_train, epochs=200, batch_size=32)

y_hat = model.predict(X_test)
y_hat = [0 if val < 0.5 else 1 for val in y_hat]

accuracy_score(y_test, y_hat)

model.save('tfmodel')

del model 
model = load_model('tfmodel')

NameError: ignored

In [ ]:
import tensorflow as tf
model = tf.keras.models.load_model('tfmodel')
import streamlit as st
import pandas as pd
import numpy as np
import cv2
import turtle

#getting the image file from the user
st.write("""
         # Pothole(s) in Road Prediction
         """
         )
st.write("This is a simple image classification web app to predict the location of potholes in an image of a road")
file = st.file_uploader("Please upload an image file", type=["jpg", "png"])


#processing the image submitted by the user
from PIL import Image, ImageOps
def import_and_predict(image_data, model):
    
        size = (150,150)    
        image = ImageOps.fit(image_data, size, Image.ANTIALIAS)
        image = np.asarray(image)
        img = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        img_resize = (cv2.resize(img, dsize=(75, 75),    interpolation=cv2.INTER_CUBIC))/255.
        
        img_reshape = img_resize[np.newaxis,...]
    
        prediction = model.predict(img_reshape)
        
        return prediction
if file is None:
    st.text("Please upload an image file")
else:
    image = Image.open(file)
    st.image(image, use_column_width=True)
    prediction = import_and_predict(image, model)

    numPotholes = model.numPotholes
    x_pos = model.x_min
    y_pos = model.y_min
    width = model.width
    height = model.height

    t = turtle.Turtle()
    t.left(270)
    t.pencolor("red")

    if np.argmax(prediction) == 0:
        st.write("There are _ potholes")
        i = 0
        while i < numPotholes:
            t.goto(x_pos, y_pos)
            t.forward(height)
            t.left(90)
            t.forward(width)
            t.left(90)
            t.forward(height)
            t.left(90)
            t.forward(width)
            t.left(90)

            x_pos = model.x_min
            y_pos = model.y_min
            width = model.width
            height = model.height

            i+=1

    elif np.argmax(prediction) == 1:
        st.write("There are no potholes")

    
    st.text("Accuracy of Guess: ")
    st.write(prediction)


#streamlit run potholes.py